In [1]:
import numpy as np
import json
from pathlib import Path
from ezc3d import c3d
from pprint import pprint
from tqdm import tqdm
import pandas as pd
import json

In [2]:
data_path = Path('/Users/wcz/Beanflows/All_Beans/swift4tf/language2motion.gt/data/')

In [3]:
c3d_folder_path = data_path/'2017-06-22 c3d/'
out_folder_path = data_path/'motion-viewer/'
meta_folder_path = data_path/'2017-06-22 meta/'

In [4]:
def get_markers(c3d_obj) -> [str]:
    markers = c3d_obj['parameters']['POINT']['LABELS']['value']
    return markers

In [5]:
def get_points_metadata(c3d_obj) -> [str]:
    points = c3d_obj['header']['points']
    return points

In [6]:
def get_marker_set(motion_num: int) -> str:
    meta_path = meta_folder_path/f'{motion_num:05}_meta.json'
    with open(str(meta_path)) as f:
        meta_dict = json.load(f)
    return meta_dict['source']['database']['identifier']

In [7]:
# get all kit or cmu label names
# from viewer.min.js
marker_connections = {
        "kit": {
            "LFHD": ["RFHD", "LBHD"],
            "RBHD": ["RFHD", "LBHD"],
            "C7": ["RFHD", "LBHD", "RBHD", "LFHD", "CLAV", "LSHO", "RSHO"],
            "STRN": ["CLAV", "RASI", "LASI"],
            "T10": ["LSHO", "RSHO"],
            "L3": ["LPSI", "RPSI", "T10"],
            "LUPA": ["LSHO", "LAEL"],
            "LAEL": ["LFRA"],
            "LFRA": ["LWTS"],
            "LWPS": ["LHPS", "LWTS"],
            "LHTS": ["LWTS", "LIFD"],
            "LHPS": ["LIFD"],
            "RUPA": ["RSHO", "RAEL"],
            "RAEL": ["RFRA"],
            "RFRA": ["RWTS"],
            "RWPS": ["RHPS", "RWTS"],
            "RHTS": ["RWTS", "RIFD"],
            "RHPS": ["RIFD"],
            "LHIP": ["LASI", "LPSI"],
            "LTHI": ["LHIP"],
            "LKNE": ["LTHI", "LTIP"],
            "LHEE": ["LTIP", "LANK", "LMT1"],
            "LMT5": ["LANK", "LTOE"],
            "LMT1": ["LTOE"],
            "RHIP": ["RASI", "RPSI"],
            "RTHI": ["RHIP"],
            "RKNE": ["RTHI", "RTIP"],
            "RHEE": ["RTIP", "RANK", "RMT1"],
            "RMT5": ["RANK", "RTOE"],
            "RMT1": ["RTOE"]
        },
        "cmu": {
            "LFHD": ["RFHD", "LBHD"],
            "RBHD": ["RFHD", "LBHD"],
            "C7": ["RFHD", "LBHD", "RBHD", "LFHD", "CLAV", "LSHO", "RSHO"],
            "STRN": ["CLAV", "RFWT", "LFWT"],
            "T10": ["LSHO", "RSHO", "LBWT", "RBWT"],
            "LBWT": ["RBWT"],
            "LUPA": ["LSHO", "LELB"],
            "LELB": ["LFRM"],
            "LFRM": ["LWRA", "LWRB"],
            "LFIN": ["LWRA", "LWRB"],
            "RUPA": ["RSHO", "RELB"],
            "RELB": ["RFRM"],
            "RFRM": ["RWRA", "RWRB"],
            "RFIN": ["RWRA", "RWRB"],
            "LTHI": ["LFWT", "LBWT"],
            "LKNE": ["LTHI", "LSHN"],
            "LHEE": ["LSHN", "LANK"],
            "LMT5": ["LANK", "LTOE"],
            "LANK": ["LTOE"],
            "RTHI": ["RFWT", "RBWT"],
            "RKNE": ["RTHI", "RSHN"],
            "RHEE": ["RSHN", "RANK"],
            "RMT5": ["RANK", "RTOE"],
            "RANK": ["RTOE"]
        }
    }

flatten = lambda l: [item for sublist in l for item in sublist]
# marker_set_labels = {m_set: set(flatten(marker_connections[m_set].values())) for m_set in ['kit', 'cmu']}
marker_set_labels = {m_set: set(flatten(marker_connections[m_set].values())).union(set(marker_connections[m_set].keys())) for m_set in ['kit', 'cmu']}

def get_label(raw_label) -> str:
    comps = raw_label.split(':')
    return comps[1] if len(comps)==2 else comps[0]        

def get_good_labels(raw_labels: [str], marker_set: str) -> [str]:
    labels = [get_label(l) for l in raw_labels]
    good_labels = [l for l in labels if l in marker_set_labels[marker_set]]
    return good_labels

In [115]:
def c3d2json(c3d_obj, motion_num: int) -> str:
    # compute interval
    points_metadata = get_points_metadata(c3d_obj)
    frame_rate = points_metadata['frame_rate']
    interval = int(1000/frame_rate)
    
    # get marker_set
    marker_set = get_marker_set(motion_num)
    
    # find mapping
    raw_labels = c3d_obj['parameters']['POINT']['LABELS']['value']
    raw_labels_clean = [get_label(l) for l in raw_labels]
    good_labels = get_good_labels(raw_labels, marker_set)
    good_label_idxs = [raw_labels_clean.index(l) for l in good_labels]
    
    # get points
    points = c3d_obj.c3d_swig.get_points()


    # transpose 2, 53, 3
    points3 = np.transpose(points[:3,:,0:], (2, 1, 0))

    # identify and remove 4 unconnected points
    good_points_mask = [idx in good_label_idxs for idx in range(points3.shape[1])]
    good_points = points3[:,good_points_mask,:]

    # reshape to 2, 53*3 and divide by 100, and round to 4 decimal places
    frames = (np.reshape(good_points, (good_points.shape[0], good_points.shape[1]*3))/100).round(4)

    # fill NaNs with column means
    col_mean = np.nanmean(frames, axis=0).round(4)
    inds = np.where(np.isnan(frames))
    frames[inds] = np.take(col_mean, inds[1])

    # format json
    frames_list = frames[:, :].tolist()

    json_out = {
        'interval': interval,
        'marker_set': marker_set,
        'markers': good_labels,
        'frames': frames_list
    }
    json_str = json.dumps(json_out) #, indent=3)
    return json_str

# save chosen metadata for all c3d files

In [ ]:
all_metadata = []
for i in tqdm(range(1, 4000)):
    c3d_path = c3d_folder_path/f'{i:05}_raw.c3d'
    if c3d_path.exists():
        # print(i, c3d_path.name, c3d_path.exists())
        c3d_obj = c3d(str(c3d_path))
        markers = get_markers(c3d_obj)
        # print(i, len(markers))
        d1 = {'sample_id': i, 'num_markers': len(markers), 'markers': markers}
        d2 = get_points_metadata(c3d_obj)
        z = {**d1, **d2}
        all_metadata.append(z)
print(len(all_metadata))
# pprint(all_metadata[:1])

In [28]:
df = pd.DataFrame(all_metadata)

In [29]:
df.to_csv(data_path/'motion-viewer-info'/'c3d_metadata.csv', index=None)

# view one c3d file metadata

In [35]:
i = 37
c3d_path = c3d_folder_path/f'{i:05}_raw.c3d'
c3d_obj = c3d(str(c3d_path))

In [36]:
c3d_obj.keys()

dict_keys(['header', 'parameters', 'data'])

In [37]:
c3d_obj['data'].keys()

dict_keys(['points', 'meta_points', 'analogs'])

In [38]:
c3d_obj['parameters'].keys()

dict_keys(['TRIAL', 'SUBJECTS', 'POINT', 'ANALOG', 'FORCE_PLATFORM', 'EVENT_CONTEXT', 'EVENT', 'MANUFACTURER', 'ANALYSIS'])

In [39]:
print(c3d_obj['parameters'])

In [40]:
get_points_metadata(c3d_obj)

{'size': 53, 'frame_rate': 100.0, 'first_frame': 0, 'last_frame': 777}

In [ ]:
c = c3d_obj

def get_params(c, level=1):
    for key in c.keys():
        value = c[key]
        print(''.join(['    ']*level), key, value)
        if hasattr(value, 'keys'):
            e = get_params(value, level=level+1)
    return None
    
get_params(c3d_obj)

# convert one motion

In [ ]:
# + convert interval
# + get marker_set
# + cleanup label names
# + filter label names
# + filter points according to marker_set

In [88]:
motion_num = 1546
c3d_path = c3d_folder_path/f'{motion_num:05}_raw.c3d'
c3d_obj = c3d(str(c3d_path))

In [89]:
raw_labels = c3d_obj['parameters']['POINT']['LABELS']['value']
len(raw_labels), raw_labels[:5]

(54,
 ['Slope:Merker1',
  'Slope:Marker2',
  'Slope:Marker3',
  'Slope:Marker4',
  'Slope:Marker5'])

In [90]:
# + cleanup label names

In [91]:
# get marker_set
marker_set = get_marker_set(motion_num)
marker_set

'kit'

In [92]:
good_labels = get_good_labels(raw_labels, marker_set)
len(good_labels), good_labels[:5]

(45, ['RPSI', 'LPSI', 'L3', 'STRN', 'T10'])

In [93]:
raw_labels_clean = [get_label(l) for l in raw_labels]

In [94]:
good_label_idxs = [raw_labels_clean.index(l) for l in good_labels]
len(good_label_idxs), good_label_idxs[:3]

(45, [8, 9, 10])

In [95]:
# get points
points = c3d_obj.c3d_swig.get_points()
points.shape

(4, 54, 607)

In [96]:
# find mapping

# transpose 2, 53, 3
points3 = np.transpose(points[:3,:,0:], (2, 1, 0))

# identify and remove 4 unconnected points
good_points_mask = [idx in good_label_idxs for idx in range(points3.shape[1])]
good_points = points3[:,good_points_mask,:]

# reshape to 2, 53*3 and divide by 100, and round to 4 decimal places
frames = (np.reshape(good_points, (good_points.shape[0], good_points.shape[1]*3))/100).round(4)

# format json
# frames_list = frames[:, :].tolist()

# fill nans with means

In [97]:
frames.shape

(607, 135)

In [ ]:
col_mean = np.nanmean(frames, axis=1).round(4)
# col_mean

In [ ]:
inds = np.where(np.isnan(frames))
# inds

In [107]:
frames[inds] = np.take(col_mean, inds[1])

In [108]:
# format json
frames_list = frames[:, :].tolist()

# convert all

In [117]:
for i in tqdm(range(1, 4000)):
    c3d_path = c3d_folder_path/f'{i:05}_raw.c3d'
    if c3d_path.exists():
        # print(i, c3d_path.name, c3d_path.exists())
        c3d_obj = c3d(str(c3d_path))
        json_str = c3d2json(c3d_obj, motion_num=i)
        out_path = out_folder_path/f'{i:05}_raw.json'
        out_path.write_text(json_str)

100%|██████████| 3999/3999 [08:30<00:00,  7.83it/s]  
